In [1]:
import numpy as np
import json
import os
import csv
import glob
import tqdm
import matplotlib.pyplot as plt
import time
import lap

In [2]:
gt_file_list = glob.glob('datasets/MOT17/train/*FRCNN/gt/gt.txt')
det_file_list = glob.glob('datasets/MOT17/train/*FRCNN/det_yolox/det.txt')
train_gt_list = glob.glob('datasets/MOT17/train/*FRCNN/gt/gt.txt')
train_seq_list = [x.replace('\\', '/').split('/')[-3] for x in train_gt_list]
seq_set = set([det.replace('\\', '/').split('/')[-3] for det in det_file_list])

In [4]:
print(train_seq_list)

In [3]:
def iou(bbox1, bbox2):
    m = len(bbox1)
    n = len(bbox2)
    ret_mat = np.zeros((m, n))
    for i in range(m):
        b1 = bbox1[i]
        a1 = (b1[2] - b1[0]) * (b1[3] - b1[1])
        for j in range(n):
            b2 = bbox2[j]
            a2 = (b2[2] - b2[0]) * (b2[3] - b2[1])
            left = max(b1[0], b2[0])
            right = min(b1[2], b2[2])
            top = max(b1[1], b2[1])
            bottom = min(b1[3], b2[3])
            if right <= left or bottom <= top:
                ret_mat[i, j] = 0
            else:
                ac = (right - left) * (bottom - top)
                ret_mat[i, j] = ac / (a1 + a2 - ac)
    
    return ret_mat

In [22]:
def evaluate(iou_t):
    true_positive_iou = []
    true_positive_conf = []
    false_positive_conf = []
    all_conf = []
    gt_cnt = 0
    fn_cnt = 0
    for seq in seq_set:
        dataset_type = 'train' if seq in train_seq_list else 'test'
        gt_file_path = os.path.join(r'P:\MOTracker\datasets\MOT17', dataset_type, seq, 'gt', 'gt.txt')
        det_file_path = os.path.join(r'P:\MOTracker\datasets\MOT17', dataset_type, seq, 'det_yolox', 'det.txt')
        gt = np.loadtxt(gt_file_path, delimiter=',')
        gt = gt[gt[:, 6] == 1]
        det = np.loadtxt(det_file_path, delimiter=',')
        all_conf.extend(det[:, 6].tolist())
        gt_cnt += len(gt)
        gt[:, (4, 5)] += gt[:, (2, 3)]
        det[:, (4, 5)] += det[:, (2, 3)]
        frame_set = set(gt[:,0].tolist()) | set(det[:,0].tolist())
        for frame in tqdm.tqdm(frame_set, desc=seq):
            gt_bboxes = gt[gt[:, 0]==frame, :]
            det_bboxes = det[det[:, 0]==frame, :]
            if len(det_bboxes) == 0:
                fn_cnt += len(gt_bboxes)
            elif len(gt_bboxes) == 0:
                false_positive_conf.extend(det_bboxes[:, 6].tolist())
                continue
            else:
                iou_mat = iou(gt_bboxes[:, 2:6], det_bboxes[:, 2:6])
                _, gt_assignment, det_assignment = lap.lapjv(1 - iou_mat, extend_cost=True, cost_limit=1 - iou_t)
                true_positive_conf.extend(det_bboxes[np.where(det_assignment != -1), 6].flatten().tolist())
                true_positive_iou.extend(iou_mat[np.arange(len(gt_bboxes))[np.where(gt_assignment!=-1)], gt_assignment[gt_assignment!=-1]].flatten().tolist())
                false_positive_conf.extend(det_bboxes[np.where(det_assignment == -1), 6].flatten().tolist())
                fn_cnt += np.sum(gt_assignment == -1)
    tp_cnt = len(true_positive_iou)
    fp_cnt = len(false_positive_conf)
    return tp_cnt / (tp_cnt + fp_cnt), tp_cnt / (tp_cnt + fn_cnt)

In [23]:
N = 10
result = np.zeros((N + 1, 2))
for idx, threshold in enumerate(np.linspace(0, 1, N + 1)):
    result[idx] = evaluate(threshold)
print(result)

In [19]:
true_positive_iou = []
true_positive_conf = []
false_positive_conf = []
all_conf = []
gt_cnt = 0
fn_cnt = 0
iou_t = 0.7
for seq in seq_set:
    dataset_type = 'train' if seq in train_seq_list else 'test'
    gt_file_path = os.path.join(r'P:\MOTracker\datasets\MOT17', dataset_type, seq, 'gt', 'gt.txt')
    det_file_path = os.path.join(r'P:\MOTracker\datasets\MOT17', dataset_type, seq, 'det_yolox', 'det.txt')
    gt = np.loadtxt(gt_file_path, delimiter=',')
    gt = gt[gt[:, 6] == 1]
    det = np.loadtxt(det_file_path, delimiter=',')
    all_conf.extend(det[:, 6].tolist())
    gt_cnt += len(gt)
    gt[:, (4, 5)] += gt[:, (2, 3)]
    det[:, (4, 5)] += det[:, (2, 3)]
    frame_set = set(gt[:,0].tolist()) | set(det[:,0].tolist())
    for frame in tqdm.tqdm(frame_set, desc=seq):
        gt_bboxes = gt[gt[:, 0]==frame, :]
        det_bboxes = det[det[:, 0]==frame, :]
        if len(det_bboxes) == 0:
            fn_cnt += len(gt_bboxes)
        elif len(gt_bboxes) == 0:
            false_positive_conf.extend(det_bboxes[:, 6].tolist())
            continue
        else:
            iou_mat = iou(gt_bboxes[:, 2:6], det_bboxes[:, 2:6])
            _, gt_assignment, det_assignment = lap.lapjv(1 - iou_mat, extend_cost=True, cost_limit=1 - iou_t)
            true_positive_conf.extend(det_bboxes[np.where(det_assignment != -1), 6].flatten().tolist())
            true_positive_iou.extend(iou_mat[np.arange(len(gt_bboxes))[np.where(gt_assignment!=-1)], gt_assignment[gt_assignment!=-1]].flatten().tolist())
            false_positive_conf.extend(det_bboxes[np.where(det_assignment == -1), 6].flatten().tolist())
            fn_cnt += np.sum(gt_assignment == -1)


In [20]:
print(true_positive_iou)

In [21]:
fig, ax = plt.subplots(2, 2, figsize=[12, 8])
ax[0, 0].set_title('TP IoU')
ax[0, 1].set_title('FP IoU')
ax[1, 0].set_title('TP Conf')
ax[1, 1].set_title('FP Conf')
ax[0, 0].hist(true_positive_iou, bins=1000)
ax[1, 0].hist(true_positive_conf, bins=1000)
ax[1, 1].hist(false_positive_conf, bins=1000)
plt.show()

In [16]:
print(len(true_positive_conf))
# print(np.mean(true_positive_iou))
# print(np.mean(true_positive_conf))
# print(np.mean(false_positive_conf))

In [27]:
print(np.percentile(true_positive_conf, 10))
print(np.percentile(false_positive_conf, 95))

In [ ]:
print(f"#TP: {len(true_positive_conf)}")
print(f"#FP: {len(false_positive_conf)}")
print(f"#FN: {fn_cnt}")
print(f"#Det: {len(all_conf)}")
print(f"#GT: {gt_cnt}")

In [3]:
import os
import numpy as np
data_dir = r'P:\MOT17\train'
seq_set = set()
detector_set = set()

In [5]:
for subdataset_name in os.listdir(data_dir):
    seq_name = '-'.join(subdataset_name.split('-')[:2])
    detector_name = subdataset_name.split('-')[-1]
    seq_set.add(seq_name)
    detector_set.add(detector_name)
for detector_name in detector_set:
    print(detector_name.center(50, '='))
    gt_cnt = 0
    det_cnt = 0
    for seq_name in seq_set:
        print(seq_name)
        subdataset_name = seq_name + '-' + detector_name
        gt_file_path = os.path.join(data_dir, subdataset_name, 'gt', 'gt.txt')
        det_file_path = os.path.join(data_dir, subdataset_name, 'det', 'det.txt')
        gt_len = len(np.loadtxt(gt_file_path, delimiter=','))
        det_len = len(np.loadtxt(det_file_path, delimiter=','))
        gt_cnt += gt_len
        det_cnt += det_len
        print(f'#GT: {gt_len}, #DET: {det_len}')
    print('TOTAL')
    print(f'#GT: {gt_cnt}, #DET: {det_cnt}')

In [1]:
import glob
import os
import re
import numpy as np
import tqdm
import matplotlib.pyplot as plt

In [7]:
size = 10000

def func(n, ori_size):
    return round(size / ori_size * n)

background = np.zeros((size, size), dtype=int)
gt_file_list = glob.glob('../datasets/Union/t*/Colon-??/gt/gt.txt')
for f in gt_file_list:
    with open(os.path.join(f, '../../seqinfo.ini')) as seqinfo_file:
        seqinfo_content = seqinfo_file.readlines()
    for srow in seqinfo_content:
        if srow.count('Width'):
            imw = int(re.search('\d+', srow).group())
        if srow.count('Height'):
            imh = int(re.search('\d+', srow).group())
    gt_content = np.loadtxt(f, delimiter=',')
    for row in tqdm.tqdm(gt_content):
        left, top, width, height = row[2:6]
        right, bottom = left + width, top + height
        left = func(left, imw)
        right = func(right, imw)
        top = func(top, imh)
        bottom = func(bottom, imh)
        background[top:bottom, left:right] += 1
plt.imshow(background, )

In [9]:
fig,ax=plt.subplots(dpi=1200)
ax.imshow(background)
fig.show()

In [12]:
ax.yaxis.set_major_locator(plt.NullLocator())
ax.xaxis.set_major_locator(plt.NullLocator())
ax.imshow(background)
fig.savefig('heatmap.pdf',format='pdf')

In [ ]:
import numpy as np

def h_jacobian(x, y, asp, h, psi, psis, v, a, vasp, vh, vpsi, vpsis):
    # 计算雅可比矩阵
    jac = np.zeros((4, 12))
    jac[0, 0] = 1  # 对 x 的偏导数
    jac[1, 1] = 1  # 对 y 的偏导数
    jac[2, 3] = np.abs(np.sin(psi)) * h  # 对 asp 的偏导数
    jac[2, 4] = h * np.abs(np.cos(psi))  # 对 h 的偏导数
    jac[2, 5] = h * asp * np.sin(psi) * np.cos(psis)  # 对 psi 的偏导数
    jac[2, 6] = np.abs(np.cos(psi))  # 对 psis 的偏导数
    jac[3, 3] = np.abs(np.cos(psi)) * h  # 对 asp 的偏导数
    jac[3, 4] = -h * np.abs(np.sin(psi))  # 对 h 的偏导数
    jac[3, 5] = -h * asp * np.cos(psi) * np.cos(psis)  # 对 psi 的偏导数
    jac[3, 6] = np.abs(np.sin(psi))  # 对 psis 的偏导数
    return jac

# 验证雅可比矩阵的满秩性
for i in range(12):
    state = np.random.rand(12)  # 生成随机状态向量
    jacobian = h_jacobian(*state)  # 计算雅可比矩阵
    rank = np.linalg.matrix_rank(jacobian)
    print(f"Rank of Jacobian at state {state}: {rank}")

# 检查观测方程是否在整个状态空间上都有定义
defined_in_all_states = True
for i in range(1000):
    state = np.random.rand(12)  # 生成随机状态向量
    observation = h(*state)  # 计算观测结果
    if np.isnan(observation).any():  # 如果观测结果中包含 NaN，则在该状态下观测方程未定义
        defined_in_all_states = False
        break

if defined_in_all_states:
    print("The observation equation is defined in the entire state space R^12.")
else:
    print("The observation equation is not defined in the entire state space R^12.")


In [ ]:
import numpy as np

In [ ]:
def pareto_sorting(x: np.ndarray, maximize=True):
    assert x.ndim == 2
    n, m = x.shape
    if not maximize:
        x = -x
    dom_ij = np.zeros((n, n), dtype=bool)
    argsort = np.zeros(n, dtype=int)
    sorted_cnt = 0
    k = 0
    for i in range(n):
        dom_ij[i, :] = np.logical_and(np.all(x[i] >= x, axis=1), np.any(x[i] > x, axis=1))
    this_tier_idx = np.where(np.all(dom_ij == False, axis=1))
    argsort[this_tier_idx] = k
    dom_ij[:, this_tier_idx] = False
    sorted_cnt += len(this_tier_idx)
    while sorted_cnt < n:
        k += 1
        this_tier_idx = np.where(np.all(dom_ij == False, axis=1))
        argsort[this_tier_idx] = k
        dom_ij[:, this_tier_idx] = False
        sorted_cnt += len(this_tier_idx)
    
    return argsort

In [ ]:
x = np.array([[1,1],[1,2],[2,1],[2,2]])
print(pareto_sorting(x))